## Urban Sound Classification using Convolutional Neural Networks with Keras: Theory and Implementation

https://medium.com/gradientcrescent/urban-sound-classification-using-convolutional-neural-networks-with-keras-theory-and-486e92785df4

The UrbanSound dataset consists of 8732 labelled short (less than 4 s) sound excerpts of urban sounds from 10 different classes. Both our training and test datasets consist of .wav files and an accompanying .csv spreadsheet detailing their ID and in the case of the training data, their correct descriptions, which serve as their labels. In order to convert our data into spectrogram representations, we will utilize LibROSA, an open-source python package for music and audio analysis.

UrbanSoundDataset
: https://urbansounddataset.weebly.com/

아래는 다른 Kaggle site에 올려진 데이터. weebly꺼 말고 이걸 받아서 사용함.
```
train.zip (3GB) 
test.zip (2GB)
```
The dataset is called UrbanSound and contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: - The dataset contains 8732 sound excerpts (<=4s) of urban sounds from 10 classes, namely: Air Conditioner Car Horn Children Playing Dog bark Drilling Engine Idling Gun Shot Jackhammer Siren Street Music The attributes of data are as follows: ID – Unique ID of sound excerpt Class – type of sound

https://www.kaggle.com/devilsknight/sound-classification-using-spectrogram-images

Source of the dataset : https://drive.google.com/drive/folders/0By0bAi7hOBAFUHVXd1JCN3MwTEU

Source of research document : https://serv.cusp.nyu.edu/projects/urbansounddataset/salamon_urbansound_acmmm14.pdf


In [7]:
import warnings
warnings.simplefilter('ignore')

In [8]:
dataloc='/home/seung/dataset/UrbanSound_devilsknight'
!ls $dataloc

test  train


In [9]:
from memory_profiler import memory_usage
#https://github.com/giampaolo/psutil/issues/1143#issuecomment-334695523
import os
import pandas as pd
from glob import glob
import numpy as np
#import tqdm as tqdm
from tqdm import tqdm_notebook as tqdm

!apt-get install libav-tools -y

In order to convert our data into spectrogram representations, we will utilize LibROSA, an open-source python package for music and audio analysis.

In [13]:
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
from path import Path

In [14]:
from DDUtil import *

In [42]:
n_classes = 1
bCreateFile=False#False
base_dir_out = '/home/seung/data/out-audio1/'
outDir = '{}/n21_CL{}'.format(base_dir_out, n_classes)
MakeDir(outDir)
MakeDir(base_dir_out+'_train')
MakeDir(base_dir_out+'_test')

print('Created {}'.format(outDir))

Created /home/seung/data/out-audio1//n21_CL1


In [20]:
!ls /home/seung/data/out-audio1

n21_CL1  n21_CL1model.h5  _test  _train


~/.venv/py3Keras/bin/pip install -I path.py==7.7.1

In [21]:
def create_spectrogram(outdir, filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    fout  = outdir + '/_train/' + name + '.jpg'
    plt.savefig(fout, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del fout,name,clip,sample_rate,fig,ax,S

In [22]:
def create_spectrogram_test(outdir, filename, name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    #fout  = Path(outdir +'/test/' + name + '.jpg')
    fout  = outdir +'/_test/' + name + '.jpg'
    fig.savefig(fout, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del fout,name,clip,sample_rate,fig,ax,S

In [23]:
Data_dir=np.array(glob(dataloc+'/train/Train/*'))
Test_dir=np.array(glob(dataloc+'/test/Test/*'))
print('Train : {}'.format(Data_dir.shape))
print('Test : {}'.format(Test_dir.shape))

Train : (5435,)
Test : (3297,)


In [24]:
Data_dir[:3]

array(['/home/seung/dataset/UrbanSound_devilsknight/train/Train/7893.wav',
       '/home/seung/dataset/UrbanSound_devilsknight/train/Train/5178.wav',
       '/home/seung/dataset/UrbanSound_devilsknight/train/Train/495.wav'],
      dtype='<U64')

In [25]:
%load_ext memory_profiler
%memit
i=0
if bCreateFile:
    with tqdm(total=len(Data_dir[i:i+2000])) as pbar:
        for file in Data_dir[i:i+2000]:
            filename,name = file,file.split('/')[-1].split('.')[0]
            create_spectrogram(base_dir_out,filename,name)
            pbar.update(1)
    gc.collect()

peak memory: 372.93 MiB, increment: 0.00 MiB


In [26]:
len(Data_dir[i:i+2000])

2000

In [27]:
%memit 
i=2000
if bCreateFile:
    with tqdm(total=len(Data_dir[i:i+2000])) as pbar:
        for file in Data_dir[i:i+2000]:
            filename,name = file,file.split('/')[-1].split('.')[0]
            create_spectrogram(base_dir_out,filename,name)
            pbar.update(1)
    gc.collect()

peak memory: 372.94 MiB, increment: 0.00 MiB


In [28]:
%memit 
i=4000
if bCreateFile:
    with tqdm(total=len(Data_dir[i:])) as pbar:
        for file in Data_dir[i:]:
            filename,name = file,file.split('/')[-1].split('.')[0]
            create_spectrogram(base_dir_out,filename,name)
            pbar.update(1)
    gc.collect()

peak memory: 372.94 MiB, increment: 0.00 MiB


Doing the same for our test dataset:

In [29]:
%memit
i=0
if bCreateFile:
    with tqdm(total=len(Test_dir[i:i+2000])) as pbar:
        for file in Test_dir[i:i+2000]:
            filename,name = file,file.split('/')[-1].split('.')[0]
            create_spectrogram_test(base_dir_out,filename,name)
            pbar.update(1)
    gc.collect()

peak memory: 372.95 MiB, increment: 0.00 MiB


In [30]:
%memit
i=2000
if bCreateFile:
    with tqdm(total=len(Test_dir[i:])) as pbar:
        for file in Test_dir[i:]:
            filename,name = file,file.split('/')[-1].split('.')[0]
            create_spectrogram_test(base_dir_out,filename,name)
            pbar.update(1)
    gc.collect()

peak memory: 372.95 MiB, increment: 0.00 MiB


In [32]:
from keras.preprocessing.image import ImageDataGenerator

def append_ext(fn):
    return fn+".jpg"

In [33]:
dataloc_test=dataloc+'/test'
dataloc_train=dataloc+'/train'

!ls $dataloc_test

Test  test.csv


In [34]:
csv_train = dataloc_train+'/train.csv'
csv_test = dataloc_test+'/test.csv'
print(csv_train)
print(csv_test)

/home/seung/dataset/UrbanSound_devilsknight/train/train.csv
/home/seung/dataset/UrbanSound_devilsknight/test/test.csv


In [35]:
traindf=pd.read_csv(csv_train,dtype=str)
testdf=pd.read_csv(csv_test,dtype=str)
traindf["ID"]=traindf["ID"].apply(append_ext)
testdf["ID"]=testdf["ID"].apply(append_ext)

In [36]:
traindf.head

<bound method NDFrame.head of             ID            Class
0        0.jpg            siren
1        1.jpg     street_music
2        2.jpg         drilling
3        3.jpg            siren
4        4.jpg         dog_bark
...        ...              ...
5430  8725.jpg    engine_idling
5431  8726.jpg         dog_bark
5432  8727.jpg    engine_idling
5433  8728.jpg    engine_idling
5434  8729.jpg  air_conditioner

[5435 rows x 2 columns]>

In [37]:
traindf["ID"]

0          0.jpg
1          1.jpg
2          2.jpg
3          3.jpg
4          4.jpg
          ...   
5430    8725.jpg
5431    8726.jpg
5432    8727.jpg
5433    8728.jpg
5434    8729.jpg
Name: ID, Length: 5435, dtype: object

In [38]:
import keras
print(keras.__version__)

2.2.5


In [40]:
#datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25)
#datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
#datagen = ImageDataGenerator(rescale=1./255)
#datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
#datagen=ImageDataGenerator(rescale=1./255.) # keras 2.1.2
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [43]:
train_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=base_dir_out+'_train',
    x_col="ID",
    y_col="Class",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

Found 4077 validated image filenames belonging to 10 classes.


In [44]:
valid_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=base_dir_out+'_train',#directory="/kaggle/working/train/",
    x_col="ID",
    y_col="Class",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

Found 1358 validated image filenames belonging to 10 classes.


In [45]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np


In [46]:

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',  input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________

In [47]:
#Fitting keras model, no test gen for now
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=150
)
model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/150
127/127 [==============================] - 7s 59ms/step - loss: 2.0896 - acc: 0.2242 - val_loss: 1.9906 - val_acc: 0.2768
Epoch 2/150
127/127 [==============================] - 5s 41ms/step - loss: 1.8140 - acc: 0.3459 - val_loss: 1.5343 - val_acc: 0.4683
Epoch 3/150
127/127 [==============================] - 5s 41ms/step - loss: 1.4883 - acc: 0.4701 - val_loss: 1.3120 - val_acc: 0.5558
Epoch 4/150
127/127 [==============================] - 5s 41ms/step - loss: 1.1846 - acc: 0.5972 - val_loss: 0.9894 - val_acc: 0.6652
Epoch 5/150
127/127 [==============================] - 5s 41ms/step - loss: 1.0037 - acc: 0.6628 - val_loss: 0.8074 - val_acc: 0.7247
Epoch 6/150
127/127 [==============================] - 5s 40ms/step - loss: 0.8206 - acc: 0.7272 - val_loss: 0.8008 - val_acc: 0.7466
Epoch 7/150
127/127 [==============================] - 5s 40ms/step - loss: 0.6986 - acc: 0.7724 - val_loss: 0.6850 - val_acc: 0.7798
Epoch 8/150
12

Epoch 60/150
127/127 [==============================] - 5s 38ms/step - loss: 0.1079 - acc: 0.9709 - val_loss: 0.4135 - val_acc: 0.9118
Epoch 61/150
127/127 [==============================] - 5s 38ms/step - loss: 0.0801 - acc: 0.9771 - val_loss: 0.3857 - val_acc: 0.9231
Epoch 62/150
127/127 [==============================] - 5s 38ms/step - loss: 0.0920 - acc: 0.9775 - val_loss: 0.4938 - val_acc: 0.9246
Epoch 63/150
127/127 [==============================] - 5s 38ms/step - loss: 0.0855 - acc: 0.9783 - val_loss: 0.5291 - val_acc: 0.9140
Epoch 64/150
127/127 [==============================] - 5s 37ms/step - loss: 0.1038 - acc: 0.9732 - val_loss: 0.3408 - val_acc: 0.9382
Epoch 65/150
127/127 [==============================] - 5s 38ms/step - loss: 0.0888 - acc: 0.9765 - val_loss: 0.3612 - val_acc: 0.9336
Epoch 66/150
127/127 [==============================] - 5s 38ms/step - loss: 0.0759 - acc: 0.9801 - val_loss: 0.4955 - val_acc: 0.9170
Epoch 67/150
127/127 [==============================] -

127/127 [==============================] - 5s 38ms/step - loss: 0.1290 - acc: 0.9723 - val_loss: 0.4590 - val_acc: 0.9419
Epoch 121/150
127/127 [==============================] - 5s 38ms/step - loss: 0.1444 - acc: 0.9744 - val_loss: 0.6466 - val_acc: 0.9246
Epoch 122/150
127/127 [==============================] - 5s 38ms/step - loss: 0.1239 - acc: 0.9763 - val_loss: 0.3660 - val_acc: 0.9284
Epoch 123/150
127/127 [==============================] - 5s 38ms/step - loss: 0.1071 - acc: 0.9803 - val_loss: 0.4607 - val_acc: 0.9367
Epoch 124/150
127/127 [==============================] - 5s 38ms/step - loss: 0.1649 - acc: 0.9716 - val_loss: 0.4260 - val_acc: 0.9246
Epoch 125/150
127/127 [==============================] - 5s 38ms/step - loss: 0.1164 - acc: 0.9773 - val_loss: 0.4354 - val_acc: 0.9118
Epoch 126/150
127/127 [==============================] - 5s 38ms/step - loss: 0.1177 - acc: 0.9776 - val_loss: 0.4474 - val_acc: 0.9457
Epoch 127/150
127/127 [==============================] - 5s 38

[0.5511467451897694, 0.9360119047619048]

In [48]:
fnmodel=outDir+'model.h5'
print(fnmodel)
model.save(fnmodel)

/home/seung/data/out-audio1//n21_CL1model.h5
